In [9]:
import re
import string
import tensorflow as tf
import random
import csv
import numpy as np
import pandas as pd


from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [11]:
!gdown 1whg44Q1UeM6d6NMuUVND2bQ6axqB2nvM
!gdown 12Xp3ouuMa1U3EFv3CY5U043ycU5QVm8T


Downloading...
From: https://drive.google.com/uc?id=1whg44Q1UeM6d6NMuUVND2bQ6axqB2nvM
To: w:\Hackaton\Országos IT megmérettetés\MI\1\text_dataset.csv

  0%|          | 0.00/489k [00:00<?, ?B/s]
100%|██████████| 489k/489k [00:00<00:00, 4.83MB/s]
100%|██████████| 489k/489k [00:00<00:00, 4.83MB/s]
Downloading...
From: https://drive.google.com/uc?id=12Xp3ouuMa1U3EFv3CY5U043ycU5QVm8T
To: w:\Hackaton\Országos IT megmérettetés\MI\1\spamtext_classifier.h5

  0%|          | 0.00/1.74M [00:00<?, ?B/s]
 30%|███       | 524k/1.74M [00:00<00:00, 4.91MB/s]
100%|██████████| 1.74M/1.74M [00:00<00:00, 12.0MB/s]


In [12]:
data = pd.read_csv('./text_dataset.csv')

In [13]:
label_dict={'legit':0, 'spam':1}

In [14]:
data['input'] = data['text'].apply(lambda x: x.replace(u'\xa0',u' '))
data['input'] = data['input'].apply(lambda x: x.replace('\u200a',' '))

In [15]:
tokenizer = Tokenizer(oov_token='<oov>') # For those words which are not found in word_index
tokenizer.fit_on_texts(data['input'])
total_words = len(tokenizer.word_index) + 1

In [16]:
data['input']=data['input'].apply(lambda x: tokenizer.texts_to_sequences([x])[0])
max_sequence_len = data['input'].str.len().max()
print(max_sequence_len)
data["input"]=data["input"].apply(lambda x: np.array(pad_sequences([x], maxlen=max_sequence_len, padding='pre')))
data["input"]=data["input"].apply(lambda x: x.squeeze())
data["label"]=data['category'].apply(lambda x: label_dict[x])


189


In [17]:
input_ds = tf.data.Dataset.from_tensor_slices((data["input"].tolist(), to_categorical(data["label"])))

In [26]:
embedding_dim = 16


model = tf.keras.Sequential([
  layers.Embedding(total_words + 1, embedding_dim),
  layers.Dropout(0.2),
  layers.GlobalAveragePooling1D(),
  layers.Dropout(0.2),
  layers.Dense(2, activation='sigmoid')])

model.compile(loss=losses.CategoricalCrossentropy(from_logits=True),
              optimizer='adam',
              metrics=tf.metrics.CategoricalAccuracy())

BATCH_SIZE = 32
input_batches = input_ds.shuffle(1000).batch(BATCH_SIZE)
epochs = 25


history = model.fit_generator(
    input_batches,
    epochs=epochs)

    


Epoch 1/25


C:\Users\tktm_\AppData\Local\Temp\ipykernel_15572\3977084260.py:20: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(
c:\Users\tktm_\anaconda3\envs\T\lib\site-packages\keras\backend.py:5531: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


175/175 [==============================] - 1s 4ms/step - loss: 0.4763 - categorical_accuracy: 0.8643
Epoch 2/25
175/175 [==============================] - 1s 4ms/step - loss: 0.3728 - categorical_accuracy: 0.8659
Epoch 3/25
175/175 [==============================] - 1s 4ms/step - loss: 0.3622 - categorical_accuracy: 0.8659
Epoch 4/25
175/175 [==============================] - 1s 4ms/step - loss: 0.3451 - categorical_accuracy: 0.8659
Epoch 5/25
175/175 [==============================] - 1s 4ms/step - loss: 0.3269 - categorical_accuracy: 0.8659
Epoch 6/25
175/175 [==============================] - 1s 4ms/step - loss: 0.3051 - categorical_accuracy: 0.8659
Epoch 7/25
175/175 [==============================] - 1s 4ms/step - loss: 0.2772 - categorical_accuracy: 0.8661
Epoch 8/25
175/175 [==============================] - 1s 4ms/step - loss: 0.2464 - categorical_accuracy: 0.8708
Epoch 9/25
175/175 [==============================] - 1s 4ms/step - loss: 0.2134 - categorical_accuracy: 0.9017
Epo

In [29]:
sentence = "Guess what? You just won a vacation to the Maldives! Cant call you right now, because im busy, but pls venmo me 100 dollars to deposit the prize, and thats it! We dont have to pay more! i will call you tomorrow first thing in the morning!"
sentence = sentence.lower() # lowercase the input sentence
sequence = tokenizer.texts_to_sequences([sentence])
padded_sequence = pad_sequences(sequence, maxlen=max_sequence_len, padding='pre')
pred = model.predict(padded_sequence)
probabilities = np.exp(pred) / np.sum(np.exp(pred), axis=0)

predicted_class = np.argmax(probabilities)

class_labels = {0: 'legit', 1: 'spam'}
predicted_label = class_labels[predicted_class]

print(predicted_label)

1/1 [==============================] - 0s 18ms/step
legit
